<a href="https://colab.research.google.com/github/tanishqbololu/PyTorch/blob/main/15_pytorch_lstm_next_word_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install nltk

  Using cached nltk-3.9.2-py3-none-any.whl.metadata (3.2 kB)
  Using cached click-8.3.1-py3-none-any.whl.metadata (2.6 kB)
Using cached nltk-3.9.2-py3-none-any.whl (1.5 MB)
Using cached click-8.3.1-py3-none-any.whl (108 kB)



[notice] A new release of pip is available: 24.2 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
from collections import Counter
from nltk.tokenize import word_tokenize
import nltk

In [ ]:
document = """About the Program
What is the course fee for  Data Science Mentorship Program (DSMP 2023)
The course follows a monthly subscription model where you have to make monthly payments of Rs 799/month.
What is the total duration of the course?
The total duration of the course is 7 months. So the total course fee becomes 799*7 = Rs 5600(approx.)
What is the syllabus of the mentorship program?
We will be covering the following modules:
Python Fundamentals
Python libraries for Data Science
Data Analysis
SQL for Data Science
Maths for Machine Learning
ML Algorithms
Practical ML
MLOPs
Case studies
You can check the detailed syllabus here - https://learnwith.campusx.in/courses/CampusX-Data-Science-Mentorship-Program-637339afe4b0615a1bbed390
Will Deep Learning and NLP be a part of this program?
No, NLP and Deep Learning both are not a part of this program’s curriculum.
What if I miss a live session? Will I get a recording of the session?
Yes all our sessions are recorded, so even if you miss a session you can go back and watch the recording.
Where can I find the class schedule?
Checkout this google sheet to see month by month time table of the course - https://docs.google.com/spreadsheets/d/16OoTax_A6ORAeCg4emgexhqqPv3noQPYKU7RJ6ArOzk/edit?usp=sharing.
What is the time duration of all the live sessions?
Roughly, all the sessions last 2 hours.
What is the language spoken by the instructor during the sessions?
Hinglish
How will I be informed about the upcoming class?
You will get a mail from our side before every paid session once you become a paid user.
Can I do this course if I am from a non-tech background?
Yes, absolutely.
I am late, can I join the program in the middle?
Absolutely, you can join the program anytime.
If I join/pay in the middle, will I be able to see all the past lectures?
Yes, once you make the payment you will be able to see all the past content in your dashboard.
Where do I have to submit the task?
You don’t have to submit the task. We will provide you with the solutions, you have to self evaluate the task yourself.
Will we do case studies in the program?
Yes.
Where can we contact you?
You can mail us at nitish.campusx@gmail.com
Payment/Registration related questions
Where do we have to make our payments? Your YouTube channel or website?
You have to make all your monthly payments on our website. Here is the link for our website - https://learnwith.campusx.in/
Can we pay the entire amount of Rs 5600 all at once?
Unfortunately no, the program follows a monthly subscription model.
What is the validity of monthly subscription? Suppose if I pay on 15th Jan, then do I have to pay again on 1st Feb or 15th Feb
15th Feb. The validity period is 30 days from the day you make the payment. So essentially you can join anytime you don’t have to wait for a month to end.
What if I don’t like the course after making the payment. What is the refund policy?
You get a 7 days refund period from the day you have made the payment.
I am living outside India and I am not able to make the payment on the website, what should I do?
You have to contact us by sending a mail at nitish.campusx@gmail.com
Post registration queries
Till when can I view the paid videos on the website?
This one is tricky, so read carefully. You can watch the videos till your subscription is valid. Suppose you have purchased subscription on 21st Jan, you will be able to watch all the past paid sessions in the period of 21st Jan to 20th Feb. But after 21st Feb you will have to purchase the subscription again.
But once the course is over and you have paid us Rs 5600(or 7 installments of Rs 799) you will be able to watch the paid sessions till Aug 2024.
Why lifetime validity is not provided?
Because of the low course fee.
Where can I reach out in case of a doubt after the session?
You will have to fill a google form provided in your dashboard and our team will contact you for a 1 on 1 doubt clearance session
If I join the program late, can I still ask past week doubts?
Yes, just select past week doubt in the doubt clearance google form.
I am living outside India and I am not able to make the payment on the website, what should I do?
You have to contact us by sending a mail at nitish.campusx@gmai.com
Certificate and Placement Assistance related queries
What is the criteria to get the certificate?
There are 2 criterias:
You have to pay the entire fee of Rs 5600
You have to attempt all the course assessments.
I am joining late. How can I pay payment of the earlier months?
You will get a link to pay fee of earlier months in your dashboard once you pay for the current month.
I have read that Placement assistance is a part of this program. What comes under Placement assistance?
This is to clarify that Placement assistance does not mean Placement guarantee. So we dont guarantee you any jobs or for that matter even interview calls. So if you are planning to join this course just for placements, I am afraid you will be disappointed. Here is what comes under placement assistance
Portfolio Building sessions
Soft skill sessions
Sessions with industry mentors
Discussion on Job hunting strategies
"""


In [ ]:
# Tokenization
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Tanishq\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Tanishq\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [ ]:
# tokenize
tokens = word_tokenize(document.lower())

In [ ]:
# build vocab
vocab = {'<UNK>':0}

for token in Counter(tokens).keys():
    if token not in vocab:
        vocab[token] = len(vocab)

In [ ]:
vocab

{'<UNK>': 0,
 'about': 1,
 'the': 2,
 'program': 3,
 'what': 4,
 'is': 5,
 'course': 6,
 'fee': 7,
 'for': 8,
 'data': 9,
 'science': 10,
 'mentorship': 11,
 '(': 12,
 'dsmp': 13,
 '2023': 14,
 ')': 15,
 'follows': 16,
 'a': 17,
 'monthly': 18,
 'subscription': 19,
 'model': 20,
 'where': 21,
 'you': 22,
 'have': 23,
 'to': 24,
 'make': 25,
 'payments': 26,
 'of': 27,
 'rs': 28,
 '799/month': 29,
 '.': 30,
 'total': 31,
 'duration': 32,
 '?': 33,
 '7': 34,
 'months': 35,
 'so': 36,
 'becomes': 37,
 '799': 38,
 '*': 39,
 '=': 40,
 '5600': 41,
 'approx': 42,
 'syllabus': 43,
 'we': 44,
 'will': 45,
 'be': 46,
 'covering': 47,
 'following': 48,
 'modules': 49,
 ':': 50,
 'python': 51,
 'fundamentals': 52,
 'libraries': 53,
 'analysis': 54,
 'sql': 55,
 'maths': 56,
 'machine': 57,
 'learning': 58,
 'ml': 59,
 'algorithms': 60,
 'practical': 61,
 'mlops': 62,
 'case': 63,
 'studies': 64,
 'can': 65,
 'check': 66,
 'detailed': 67,
 'here': 68,
 '-': 69,
 'https': 70,
 '//learnwith.campusx.i

In [ ]:
len(vocab)

289

In [ ]:
# extract sentences from data
input_sentences = document.split('\n')

In [ ]:
input_sentences

['About the Program',
 'What is the course fee for  Data Science Mentorship Program (DSMP 2023)',
 'The course follows a monthly subscription model where you have to make monthly payments of Rs 799/month.',
 'What is the total duration of the course?',
 'The total duration of the course is 7 months. So the total course fee becomes 799*7 = Rs 5600(approx.)',
 'What is the syllabus of the mentorship program?',
 'We will be covering the following modules:',
 'Python Fundamentals',
 'Python libraries for Data Science',
 'Data Analysis',
 'SQL for Data Science',
 'Maths for Machine Learning',
 'ML Algorithms',
 'Practical ML',
 'MLOPs',
 'Case studies',
 'You can check the detailed syllabus here - https://learnwith.campusx.in/courses/CampusX-Data-Science-Mentorship-Program-637339afe4b0615a1bbed390',
 'Will Deep Learning and NLP be a part of this program?',
 'No, NLP and Deep Learning both are not a part of this program’s curriculum.',
 'What if I miss a live session? Will I get a recording 

In [ ]:
def text_to_indices(sentence, vocab):
    numerical_sentence = []

    for token in sentence:
        if token in vocab:
            numerical_sentence.append(vocab[token])
        else:
            numerical_sentence.append(vocab['<UNK>'])

    return numerical_sentence

In [ ]:
input_numerical_sentences = []

for sentence in input_sentences:
   input_numerical_sentences.append(text_to_indices(word_tokenize(sentence.lower()), vocab))

In [ ]:
len(input_numerical_sentences)

78

In [ ]:
# creating training sequences (prefix ngrams)
training_sequence = []
for sentence in input_numerical_sentences:

    for i in range(1, len(sentence)):
        training_sequence.append(sentence[:i+1])

In [ ]:
training_sequence

[[1, 2],
 [1, 2, 3],
 [4, 5],
 [4, 5, 2],
 [4, 5, 2, 6],
 [4, 5, 2, 6, 7],
 [4, 5, 2, 6, 7, 8],
 [4, 5, 2, 6, 7, 8, 9],
 [4, 5, 2, 6, 7, 8, 9, 10],
 [4, 5, 2, 6, 7, 8, 9, 10, 11],
 [4, 5, 2, 6, 7, 8, 9, 10, 11, 3],
 [4, 5, 2, 6, 7, 8, 9, 10, 11, 3, 12],
 [4, 5, 2, 6, 7, 8, 9, 10, 11, 3, 12, 13],
 [4, 5, 2, 6, 7, 8, 9, 10, 11, 3, 12, 13, 14],
 [4, 5, 2, 6, 7, 8, 9, 10, 11, 3, 12, 13, 14, 15],
 [2, 6],
 [2, 6, 16],
 [2, 6, 16, 17],
 [2, 6, 16, 17, 18],
 [2, 6, 16, 17, 18, 19],
 [2, 6, 16, 17, 18, 19, 20],
 [2, 6, 16, 17, 18, 19, 20, 21],
 [2, 6, 16, 17, 18, 19, 20, 21, 22],
 [2, 6, 16, 17, 18, 19, 20, 21, 22, 23],
 [2, 6, 16, 17, 18, 19, 20, 21, 22, 23, 24],
 [2, 6, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25],
 [2, 6, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 18],
 [2, 6, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 18, 26],
 [2, 6, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 18, 26, 27],
 [2, 6, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 18, 26, 27, 28],
 [2, 6, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 18

In [ ]:
len(training_sequence)

942

these numeircal lists will our input for lstm, but the problem here is each one has different size --> lets do padding



In [ ]:
len_list = []
for seq in training_sequence:
    len_list.append(len(seq))

max(len_list) # match other seq to 62 size with padding

62

In [ ]:
# zero pre-padding
padded_training_sequence = []
for sequence in training_sequence:
    padded_training_sequence.append([0]*(max(len_list)-len(sequence)) + sequence)

In [ ]:
padded_training_sequence[0]

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 2]

In [ ]:
len(padded_training_sequence[0])

62

In [ ]:
padded_training_sequence = torch.tensor(padded_training_sequence, dtype=torch.long)

In [ ]:
padded_training_sequence.shape

torch.Size([942, 62])

In [ ]:
X = padded_training_sequence[:,:-1]
y = padded_training_sequence[:,-1]

In [ ]:
X

tensor([[  0,   0,   0,  ...,   0,   0,   1],
        [  0,   0,   0,  ...,   0,   1,   2],
        [  0,   0,   0,  ...,   0,   0,   4],
        ...,
        [  0,   0,   0,  ...,   0, 285, 176],
        [  0,   0,   0,  ..., 285, 176, 286],
        [  0,   0,   0,  ..., 176, 286, 287]])

In [ ]:
y

tensor([  2,   3,   5,   2,   6,   7,   8,   9,  10,  11,   3,  12,  13,  14,
         15,   6,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  18,  26,
         27,  28,  29,  30,   5,   2,  31,  32,  27,   2,   6,  33,  31,  32,
         27,   2,   6,   5,  34,  35,  30,  36,   2,  31,   6,   7,  37,  38,
         39,  34,  40,  28,  41,  12,  42,  30,  15,   5,   2,  43,  27,   2,
         11,   3,  33,  45,  46,  47,   2,  48,  49,  50,  52,  53,   8,   9,
         10,  54,   8,   9,  10,   8,  57,  58,  60,  59,  64,  65,  66,   2,
         67,  43,  68,  69,  70,  50,  71,  72,  58,  73,  74,  46,  17,  75,
         27,  76,   3,  33,  78,  74,  73,  72,  58,  79,  80,  81,  17,  75,
         27,  76,   3,  82,  83,  84,  30,  85,  86,  87,  17,  88,  89,  33,
         45,  86,  90,  17,  91,  27,   2,  89,  33,  93,  94,  95,  80,  96,
         78,  36,  97,  85,  22,  87,  17,  89,  22,  65,  98,  99,  73, 100,
          2,  91,  30,  65,  86, 101,   2, 102, 103,  33,  76, 1

In [ ]:
class CustomDataset(Dataset):

    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return self.X.shape[0]

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [ ]:
dataset = CustomDataset(X, y)

In [ ]:
dataset[0]

(tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]),
 tensor(2))

In [ ]:
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [ ]:
X.shape

torch.Size([942, 61])

32 seqs, each seq with 61 words, words converted to  100d vectors

In [ ]:
class LSTMModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, 100)
        self.lstm = nn.LSTM(100, 150, batch_first=True)
        self.fc = nn.Linear(150, vocab_size)

    def forward(self, x):
        embedded = self.embedding(x)
        intermediate_hidden_state, (final_hidden_state, final_cell_state) = self.lstm(embedded)
        output = self.fc(final_hidden_state.squeeze(0))
        return output


In [ ]:
model = LSTMModel(len(vocab))

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
device

device(type='cuda')

In [ ]:
model.to(device)

LSTMModel(
  (embedding): Embedding(289, 100)
  (lstm): LSTM(100, 150, batch_first=True)
  (fc): Linear(in_features=150, out_features=289, bias=True)
)

In [ ]:
epochs = 100
learning_rate = 0.001

criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# training loop

for epoch in range(epochs):
  total_loss = 0

  for batch_x, batch_y in dataloader:

    batch_x, batch_y = batch_x.to(device), batch_y.to(device)

    optimizer.zero_grad()

    output = model(batch_x)

    loss = criterion(output, batch_y)

    loss.backward()

    optimizer.step()

    total_loss = total_loss + loss.item()

  print(f"Epoch: {epoch + 1}, Loss: {total_loss:.4f}")

Epoch: 1, Loss: 165.8116
Epoch: 2, Loss: 147.0957
Epoch: 3, Loss: 134.4321
Epoch: 4, Loss: 122.0077
Epoch: 5, Loss: 109.3067
Epoch: 6, Loss: 97.5612
Epoch: 7, Loss: 87.1727
Epoch: 8, Loss: 76.7235
Epoch: 9, Loss: 67.4677
Epoch: 10, Loss: 59.5029
Epoch: 11, Loss: 52.3011
Epoch: 12, Loss: 45.8928
Epoch: 13, Loss: 39.9662
Epoch: 14, Loss: 35.1369
Epoch: 15, Loss: 30.9295
Epoch: 16, Loss: 26.8394
Epoch: 17, Loss: 23.6178
Epoch: 18, Loss: 21.0153
Epoch: 19, Loss: 18.7044
Epoch: 20, Loss: 16.8131
Epoch: 21, Loss: 14.9581
Epoch: 22, Loss: 13.4820
Epoch: 23, Loss: 12.2601
Epoch: 24, Loss: 11.3687
Epoch: 25, Loss: 10.5054
Epoch: 26, Loss: 9.7996
Epoch: 27, Loss: 9.1722
Epoch: 28, Loss: 8.5266
Epoch: 29, Loss: 7.9898
Epoch: 30, Loss: 7.7794
Epoch: 31, Loss: 7.2478
Epoch: 32, Loss: 7.1513
Epoch: 33, Loss: 6.7903
Epoch: 34, Loss: 6.3694
Epoch: 35, Loss: 6.1246
Epoch: 36, Loss: 5.8542
Epoch: 37, Loss: 5.6455
Epoch: 38, Loss: 5.6226
Epoch: 39, Loss: 5.4197
Epoch: 40, Loss: 5.2979
Epoch: 41, Loss: 5.

In [ ]:
# prediction top 1

def prediction(model, vocab, text):

    # text tokenize
    tokenized_test = word_tokenize(text.lower())

    # text -> numerical indices
    numerical_text = text_to_indices(tokenized_test, vocab)

    # padding
    padded_text = torch.tensor([0]*(61-len(numerical_text)) + numerical_text, dtype=torch.long).unsqueeze(0).to(device)

    # send to model
    output = model(padded_text)

    # predicted index
    value, index = torch.max(output, dim=1)
    #list(vocab.keys())[index]

    # merge with text
    return text + " " + list(vocab.keys())[index]


In [ ]:
prediction(model, vocab, "the course follows")

'the course follows a'

In [ ]:
import time

num_tokens = 12
input_text = "the course follows"

for i in range(num_tokens):
    output_text = prediction(model, vocab, input_text)
    print(output_text)
    input_text = output_text

    time.sleep(0.5) # just for visual

the course follows a
the course follows a monthly
the course follows a monthly subscription
the course follows a monthly subscription model
the course follows a monthly subscription model where
the course follows a monthly subscription model where you
the course follows a monthly subscription model where you have
the course follows a monthly subscription model where you have to
the course follows a monthly subscription model where you have to make
the course follows a monthly subscription model where you have to make monthly
the course follows a monthly subscription model where you have to make monthly payments
the course follows a monthly subscription model where you have to make monthly payments of


In [ ]:
# prediction top k

def predictionk(model, vocab, text, top_k):

    # text tokenize
    tokenized_test = word_tokenize(text.lower())

    # text -> numerical indices
    numerical_text = text_to_indices(tokenized_test, vocab)

    # padding
    padded_text = torch.tensor([0]*(61-len(numerical_text)) + numerical_text, dtype=torch.long).unsqueeze(0).to(device)

    # send to model
    output = model(padded_text)

    # predicted top-k indices
    value, indices = torch.topk(output, dim=1, k=top_k)
    #list(vocab.keys())[index]

    # convert indices -> words
    vocab_list = list(vocab.keys())
    indices = indices.squeeze(0)
    preds = [vocab_list[i] for i in indices]

    return text + "->" +",".join(preds)


In [ ]:
import time

num_tokens = 21
input_text = "Maths for Machine Learning"

for i in range(num_tokens):
    output_text = predictionk(model, vocab, input_text, 3)
    print(output_text)
    input_text = output_text

    time.sleep(0.5) # just for visual

Maths for Machine Learning->learning,and,both
Maths for Machine Learning->learning,and,both->are,not,you
Maths for Machine Learning->learning,and,both->are,not,you->will,can,make
Maths for Machine Learning->learning,and,both->are,not,you->will,can,make->the,i,payment
Maths for Machine Learning->learning,and,both->are,not,you->will,can,make->the,i,payment->.,you,past
Maths for Machine Learning->learning,and,both->are,not,you->will,can,make->the,i,payment->.,you,past->can,you,paid
Maths for Machine Learning->learning,and,both->are,not,you->will,can,make->the,i,payment->.,you,past->can,you,paid->us,payment,videos
Maths for Machine Learning->learning,and,both->are,not,you->will,can,make->the,i,payment->.,you,past->can,you,paid->us,payment,videos->i,on,you
Maths for Machine Learning->learning,and,both->are,not,you->will,can,make->the,i,payment->.,you,past->can,you,paid->us,payment,videos->i,on,you->can,will,don
Maths for Machine Learning->learning,and,both->are,not,you->will,can,make->the,i